# Prime Factorization with Shor's Algorithm

# Theory

## Preliminary Facts and Variables
`N`: number to factor \
`a`: number coprime to `N` \
`r`: order of `a` \
`K`: `GCD(a,N)` \
`n`: number of qubits required, as a function of `n = lg(N-1)`

## Contents

1. [Classical Reduction](#classical-reduction)
2. [Quantum Order-Finding Kernel](#quantum-order-finding-kernel)

## Classical Reduction
Goal: find a number `1 < a < N` that is coprime to `N`

The streamlined Shor's algorithm assumes two things: `N` is odd and not a prime power (can be represented as a positive integer power of a prime number). These are basic assumptions since either of these conditions being false would mean that N is not prime factorable.

The classical reduction mainly consists of generating a random `a`, finding `K = GCD(a,N)`, and then calling the quantum order-finding kernel. If `K != 1`, then `K` is a nontrivial factor of `N` and the other factor is `N/K`, and we have successfully found nontrivial factors of `N`. If `K = 1`, we call the quantum kernel to find order `r` of `a`. Due to the way the problem is broken down, `r` must be even; if it isn't, then we restart and find a new `a`.

Once returning from the quantum kernel with an even `r`, we calculate $x = a^\frac{r}{2} - 1$ and `d = GCD(x,N)`. If `d > 1`, then it is one of our factors, with `N/d` being the other.


## Quantum Order Finding Kernel
Goal: find the order `r` of a number `a`

The bulk of this kernel is a function M<sub>a</sub> which is defined as the `n`-qubit operation M<sub>a</sub>|x> s/t:

$$
M_a |x⟩ =
\begin{cases}
|ax (mod N)⟩ & \quad 0 \leq x < N\\
|x⟩ & \quad N \leq x < 2^n
\end{cases}
$$

This can be done by building a circuit $|x⟩|y⟩ \to |x⟩|y \oplus f_a(x)⟩$, which has size $O(n^2)$.

The general form of the quantum kernel is as follows:
1. Prepare `n` control qubits into a uniform superposition by applying Hadamard gates to each control qubit.
2. Prepare `n` target qubits into the state `|1⟩`.
3. Iteratively apply $M_a ^ {2^i}$ where $i = 0, 1, \cdots , {2n-1}$ using each of the control qubits to control the operation.
4. Apply the Inverse QFT
5. Measure to get a random integer of the form $\frac{j}{r} 2^{2n}$
6. Carry out continued fractions algorithm to extract `r`










# General Process for Shors(N)

Given a number `N`, find two factors F and G.

1. Pick a random `a`
2. Compute `K = gcd(a,N)`
3. If `K != 1`, then $F = K$ and $G = N/K$
4. Call the quantum kernel on `a`
5. If `r` is odd, restart at step 1
6. Compute $G = GCD(N, a^{\frac{r}{2}} + 1)$
7. If $G$ is nontrivial, we have $G$ and $F = N/G$

# Implementation

The biggest challenge in implementing Shor's efficiently is exponentiating $M_a$ to get $M_b = M_a^{2^i}$



In [4]:
!pip install cudaq
import numpy as np
import cudaq




In [5]:
# Global variables

N = 15

REGSIZE = int(np.ceil(np.log2(N)))
CONTROL = cudaq.qvector(REGSIZE*2)
TARGET = cudaq.qvector(REGSIZE)

print(f"N = {N}")
print(f"REGSIZE = {REGSIZE}")
# print("CONTROL:")
# print(cudaq.get_state(CONTROL))
# print("TARGET:")
# print(cudaq.get_state(TARGET))

A = -1
R = -1

F = G = -1

N = 15
REGSIZE = 4


In [3]:
# Initialize the system
def init():
  """
  Initializes our system with the two registers

  First register (CONTROL) is initialized as a uniform superposition of |0⟩ states
  Second register (TARGET) is initialized as n |1> states
  """

  for i in range(2*REGSIZE):
    if i < REGSIZE:
      cudaq.x(TARGET[i])
    cudaq.h(CONTROL[i])

  print("CONTROL:")
  print(cudaq.get_state(CONTROL))
  print("\nTARGET:")
  print(cudaq.get_state(TARGET))

In [ ]:
# Modular exponentiation of M_a

def mod_exp_Ma(a, j):
  """
  Calculates M_a^j using repeated modular multiplication (assuming we use small enough examples so as not to blow up my copmuter)

  Args:
      a: our random guess
      j: power to raise M_a to

  Returns:
      None: Modifies registers
  """

# Sources

Classical Reduction -
[Shor's Algorithm (IBM)](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/phase-estimation-and-factoring#shors-algorithm)